### Jade's Changes

<b>Current problems</b>:
- amp;
- div &gt; div.group &gt; p:first-child"&gt; = video cnbc
- no space between words separated by new line
- Check out the companies making headlines before the belldiv gt divgroup gt pfirstchildgt
- CNBC in general
- shit at the end: To contact... blah blah
- © CNBC is a USA TODAY content partner offering financial news and commentary. Its content is produced independently of USA TODAY.
- Read or Share this story:
- Read more here.
- FT subscribers can click here to receive Market Forces every day by email.
- Your feedback
- I'd love to hear from you. You can email me on michael.mackenzie@ft.com and follow me on Twitter at @michaellachlan.
- COPYRIGHT
- © 2019 Bloomberg L.P
- (Bloomberg)—
- Anything with "Transcript" should be removed
- Write to Sara Castellanos at sara.castellanos@wsj.com
- Disclosure: Cramer's charitable trust owns shares of Johnson &amp; Johnson.
- Questions for Cramer?Call Cramer: 1-800-743-CNBC
- Want to take a deep dive into Cramer's world? Hit him up!Mad Money Twitter - Jim Cramer Twitter - Facebook - Instagram
- Questions, comments, suggestions for the "Mad Money" website? madcap@cnbc.com
- RPT-

<b>Some rules I'm considering</b>:
- Get rid of CNBC as a source (optional)
- Get rid of articles with less than 5 periods
- Get rid of articles where writers say "I" a lot
- Periods between 2 numbers should be preserved
- Remove any lines in articles that don't contain periods
- Remove any lines that don't have more than 5 words
- Remove sentences containing "more here"
- get rid of anything with Jim Cramer


In [20]:
def DataClean(filename):
    #import FeatureEncoding
    import os
    from os import listdir
    from os.path import isfile, join
    from pathlib import Path
    import pandas as pd
    import FeatureEncoding
    import re, string

    # Import Article Data including corresponding Y values
    DATA_DIR = "Data"    
    RAW_DIR = os.path.join(DATA_DIR, filename)
    raw = pd.read_excel(RAW_DIR)

    #remove blanks (NaN)
    df = raw.dropna(subset = ['content', 'title']) 

    #remove blocked articles without content
    df = df[df.content.str.contains("Your usage has been flagged") == False]
    df = df[df.content.str.contains("To continue, please click the box") == False]

    #remove duplicates by url
    df = df.drop_duplicates(subset=['url'], keep='first')

    #remove duplicates by content
    df = df.drop_duplicates(subset=['content'], keep='first')

    #remove punctuation 
    pattern = re.compile('[^0-9a-zA-Z ]+')
    content= map(lambda x: pattern.sub(' ', x), df['content'])
    df['content']=list(content)

    # Output Cleaned Article Data
    OUTPUT_DIR = os.path.join(DATA_DIR, "cleanedArticles.csv")
    pd.DataFrame.to_csv(df, path_or_buf=OUTPUT_DIR)

In [21]:
def test():
    filename = "sortedarticlesRetail-List-1.xlsx"
    DataClean(filename)

In [22]:
#test()